In [6]:
!pip install -q pyomo

In [7]:
from pyomo.environ import * 

In [8]:
import numpy as np

In [9]:
coef = np.loadtxt('lab4_ex1_coef .txt', delimiter=',')

In [10]:
print(coef.shape)
print('Num rows:',coef.shape[0])
print('Num cols:',coef.shape[1])


(6, 7)
Num rows: 6
Num cols: 7


In [11]:
print(coef)

[[  0.65   0.75  -5.85  -8.35 -12.38  30.     0.  ]
 [ -0.5    0.    -3.8   -0.9    0.9    0.    24.8 ]
 [  1.     0.     0.    -1.     0.    -1.     0.8 ]
 [  0.    -2.9   -1.9    0.     0.    -0.9  -71.4 ]
 [  0.     0.     0.     4.7    8.02   6.15  95.  ]
 [ -0.7   -1.4    7.9   -2.9    0.9    0.   -47.  ]]


In [12]:
# create a model
model_lab4 = ConcreteModel()

In [13]:
# we can set certain values using python variables
N = coef.shape[1]-1 #we declare a Python variable N denoting the number of decision variables  
#Note: We subtract a one since the last column corresponds to the scalar (or constraint RHS)

In [14]:
M = coef.shape[0]-1 #we declare a Python variable M denoting the number of constraints
#Note: We subtract a one since the first row corresponds to the objective function. 

In [15]:
# coefficients for objective function can be obtained by accessing the coef array
obj_coef = coef[0,:-1]
print(obj_coef.shape)
print(obj_coef)


(6,)
[  0.65   0.75  -5.85  -8.35 -12.38  30.  ]


In [16]:
# We can segregate the coefficients for constraints into a separate array
constr_coef = coef[1:,:-1]
print(constr_coef.shape)
print(constr_coef)

(5, 6)
[[-0.5   0.   -3.8  -0.9   0.9   0.  ]
 [ 1.    0.    0.   -1.    0.   -1.  ]
 [ 0.   -2.9  -1.9   0.    0.   -0.9 ]
 [ 0.    0.    0.    4.7   8.02  6.15]
 [-0.7  -1.4   7.9  -2.9   0.9   0.  ]]


In [17]:
# We can segregate the RHS of constraints into a separate array
constr_rhs = coef[1:,-1]
print(constr_rhs.shape)
print(constr_rhs)

(5,)
[ 24.8   0.8 -71.4  95.  -47. ]


In [18]:
#Since the variables have the same upper bound and lower bound, we can simply use scalars to store them
lower_bound = 0
upper_bound = np.inf

In [19]:
# set of column indices: since we have N variables, we can take column indices to be the list [0,1,2,...,N-1]
col_indices = np.arange(N)
print(col_indices)

[0 1 2 3 4 5]


In [20]:
#declare the decision variables in the model
model_lab4.x = Var(col_indices,domain=NonNegativeReals)

In [21]:
# set of row indices: since we have M contraints, we can take row indices to be the list [0,1,2,...,M-1]
row_indices = np.arange(M)
print(row_indices)

[0 1 2 3 4]


In [22]:
# create a ConstraintList to hold multiple constraints
model_lab4.constraints = ConstraintList()

In [23]:
for i in row_indices:
  model_lab4.constraints.add(summation(constr_coef[i],model_lab4.x) <= constr_rhs[i])

In [24]:
#add the upper bound and lower bounds for the variables
for j in col_indices:
  model_lab4.x[j].setlb(lower_bound)
  model_lab4.x[j].setub(upper_bound)

In [25]:
# add the model objective
model_lab4.objective = Objective(expr = summation(obj_coef,model_lab4.x), sense=maximize)

In [26]:
model_lab4.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
          5 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
       

In [27]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [28]:
opt_cbc = SolverFactory('cbc')

In [29]:
result = opt_cbc.solve(model_lab4)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: unbounded
  - message from solver: <undefined>


Solver status: warning
Solver termination condition: unbounded


In [30]:
# reset the sense of the model objective
model_lab4.objective.set_sense(minimize)

In [31]:
model_lab4.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
          5 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
       

In [32]:
result = opt_cbc.solve(model_lab4)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: unbounded
  - message from solver: <undefined>


Solver status: warning
Solver termination condition: unbounded


In [33]:
model_lab4.x[1].setub(100)

In [34]:
model_lab4.objective.set_sense(maximize)

In [35]:
model_lab4.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :   100 : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
          5 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
       

In [36]:
result = opt_cbc.solve(model_lab4)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [37]:
# display solution
print('\nObjective = ', model_lab4.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_lab4.x[i].value)

print('\nConstraints')
model_lab4.constraints.display()


Objective =  548.9752701

Decision Variables
x[ 0 ] =  16.247154
x[ 1 ] =  100.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0
x[ 5 ] =  15.447154

Constraints
constraints : Size=5
    Key : Lower : Body               : Upper
      1 :  None :          -8.123577 :  24.8
      2 :  None : 0.7999999999999989 :   0.8
      3 :  None :       -303.9024386 : -71.4
      4 :  None :         94.9999971 :  95.0
      5 :  None :       -151.3730078 : -47.0


Answer 1>7>
Solver termination condition: unbounded
Solver status: warning
There is no bound given to x2 and hence it is shooting up to infinity then solution unbounded or may be any constraint is incorrect 



9>We changed the objective function to minimize from maximize but doing this still gives unbounded condition 
if the value of upper bound of x2 is below 25 then we do not get any solution. so when we give an upper bound to the variable x2 we get the solution with lower iterations. 